In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r'C:\Users\boyu571\boyu571_Github\01_Kakaobank_SKKU_Research_23\data\foreign_news_data\eng_news_nocontent.csv')
df

,source,author,title,description,url,urlToImage,publishedAt,content,press
0,"{'id': None, 'name': 'Trendmicro.com'}",Trend Micro,REDBALDKNIGHT/BRONZE BUTLER’s Daserf Backdoor ...,"REDBALDKNIGHT, also known as BRONZE BUTLER and...",https://blog.trendmicro.com/trendlabs-security...,https://blog.trendmicro.com/trendlabs-security...,2017-11-07T12:34:44Z,by Joey Chen and MingYen Hsieh (Threat Analyst...,Trendmicro.com
1,"{'id': None, 'name': 'Computerworld'}","Lucas Mearian, Lucas Mearian",What is blockchain? The most disruptive tech i...,Blockchain is poised to change IT in much the ...,https://www.computerworld.com/article/3191077/...,https://images.techhive.com/images/article/201...,2017-11-08T01:06:00Z,Blockchain is poised to change IT in much the ...,Computerworld
2,"{'id': None, 'name': 'hackernoon.com'}",Yaoqi Jia,Demystifying Hashgraph: Benefits and Challenges,"Recently, on our Zilliqa social channels a num...",https://hackernoon.com/demystifying-hashgraph-...,NaN,2017-11-08T17:21:35Z,"Recently, on our Zilliqa social channels a num...",hackernoon.com
3,"{'id': None, 'name': 'Topix.com'}",NaN,BitTorrent inventor announces eco-friendly Bit...,A Bitcoin transaction wastes as much electrici...,http://www.topix.com/tech/p2p/2017/11/bittorre...,http://picproxy.topixcdn.com/pics/up-,2017-11-08T23:21:17Z,A Bitcoin transaction wastes as much electrici...,Topix.com
4,"{'id': None, 'name': 'Ml-implode.com'}",NaN,Bitcoin Fork Called Off: Prices Soar After Seg...,The price of a bitcoin went on a tear on Wedne...,http://ml-implode.com/staticnews/2017-11-09_Bi...,NaN,2017-11-09T00:00:00Z,The price of a bitcoin went on a tear on Wedne...,Ml-implode.com
...,...,...,...,...,...,...,...,...,...
744427,"{'id': None, 'name': 'Biztoc.com'}",u.today,"Negatively Correlates With Bitcoin (BTC), What...",Shiba Inu continues to focus on utility as its...,https://biztoc.com/x/8248d6342c37d743,https://c.biztoc.com/p/8248d6342c37d743/s.webp,2023-09-30T21:50:10Z,Shiba Inu continues to focus on utility as its...,Biztoc.com
744428,"{'id': None, 'name': 'Slashdot.org'}",feedfeeder,Industry leaders and policymakers weigh in on ...,Industry leaders and policymakers weigh in on ...,https://slashdot.org/firehose.pl?op=view&amp;i...,NaN,2023-09-30T21:53:20Z,Sign up for the Slashdot newsletter! OR check ...,Slashdot.org
744429,"{'id': None, 'name': 'Investing.com'}",U.Today,Shiba Inu (SHIB) Negatively Correlates With Bi...,Shiba Inu (SHIB) Negatively Correlates With Bi...,https://www.investing.com/news/cryptocurrency-...,https://i-invdn-com.investing.com/news/LYNXNPE...,2023-09-30T22:00:22Z,U.Today - Dog-themed cryptocurrency is seeing ...,Investing.com
744430,"{'id': None, 'name': 'Ambcrypto.com'}",Benjamin Njiri,Can NEAR stay above $1?,"NEAR could be ripe for a price reversal, but t...",https://ambcrypto.com/?p=347645,https://statics.ambcrypto.com/wp-content/uploa...,2023-09-30T22:30:04Z,Disclaimer: The information presented does not...,Ambcrypto.com


In [291]:
df[df['press'] == 'Tech Times']['url'].iloc[0]

'http://www.techtimes.com/articles/215983/20171121/cryptocurrency-startup-tether-hacked-attackers-take-31-million-worth-of-bitcoin.htm'

In [293]:
TechTimes = df[df['press'] == 'Tech Times']
TechTimes

,source,author,title,description,url,urlToImage,publishedAt,content,press
3412,"{'id': None, 'name': 'Tech Times'}",NaN,Cryptocurrency Startup Tether Hacked: Attacker...,"Tether, a cryptocurrency startup, has lost $31...",http://www.techtimes.com/articles/215983/20171...,http://image.techtimes.com/data/images/full/31...,2017-11-21T14:57:36Z,How cryptocurrency is changing the world Nearl...,Tech Times
6330,"{'id': None, 'name': 'Tech Times'}",NaN,Amazon Sumerian Is A VR App Creator For The VR...,Amazon has unveiled a web-based toolkit called...,http://www.techtimes.com/articles/216283/20171...,http://image.techtimes.com/data/images/full/31...,2017-11-29T11:18:04Z,The five craziest bids for Amazon's new HQ Whe...,Tech Times
8378,"{'id': None, 'name': 'Tech Times'}",NaN,Elon Musk Denies He Is Bitcoin Inventor Satosh...,Elon Musk as the creator of bitcoins steers up...,http://www.techtimes.com/articles/216422/20171...,http://image.techtimes.com/data/images/full/31...,2017-12-02T12:11:39Z,How cryptocurrency is changing the world The r...,Tech Times
8801,"{'id': None, 'name': 'Tech Times'}",NaN,Cryptocurrency Owners Have Already Spent Almos...,Tech Times Cryptocurrency Owners Have Already ...,http://www.techtimes.com/articles/216570/20171...,http://image.techtimes.com/data/images/full/31...,2017-12-04T03:53:09Z,How cryptocurrency is changing the world Crypt...,Tech Times
24214,"{'id': None, 'name': 'Tech Times'}",NaN,Bitcoin's Price Drops Sending Panic In The Cry...,Cryptocurrency is a volatile investment. With ...,http://www.techtimes.com/articles/217461/20171...,http://www.techtimes.com//image.techtimes.com/...,2017-12-25T05:17:47Z,How cryptocurrency is changing the world Bitco...,Tech Times
...,...,...,...,...,...,...,...,...,...
646798,"{'id': None, 'name': 'Tech Times'}",John Lopez,South Korea Invests $18 Million in New 'Metave...,The South Korean Ministry of Science and ICT h...,https://www.techtimes.com/articles/288860/2023...,https://1734811051.rsc.cdn77.org/data/images/f...,2023-03-12T18:09:36Z,South Korea's Ministry of Science and ICT rece...,Tech Times
646976,"{'id': None, 'name': 'Tech Times'}",Urian B.,Afun Interactive Gets $7.6 Million in Series A...,Afun Interactive recently raised $7.6 million ...,https://www.techtimes.com/articles/288877/2023...,https://1734811051.rsc.cdn77.org/data/images/f...,2023-03-13T05:10:00Z,"Afun Interactive, a Korean startup specializin...",Tech Times
648919,"{'id': None, 'name': 'Tech Times'}",John Lopez,Cybercriminals' Favorite Crypto Platform ChipM...,Criminals use tools like ChipMixer to make lau...,https://www.techtimes.com/articles/289005/2023...,https://1734811051.rsc.cdn77.org/data/images/f...,2023-03-15T17:10:00Z,"In a landmark effort against cybercrime, globa...",Tech Times
652254,"{'id': None, 'name': 'Tech Times'}",John Lopez,GameStop's Quarterly Profit Results in 40% Sur...,GameStop's shares rose 40% in premarket trade ...,https://www.techtimes.com/articles/289334/2023...,https://1734811051.rsc.cdn77.org/data/images/f...,2023-03-22T12:28:05Z,GameStop shares soared 40% in premarket trade ...,Tech Times


In [282]:

url = 'http://www.thestar.com.my/business/business-news/2017/11/13/asia-shares-down-on-caution-over-us-tax-reform-plan-sterling-falls/'
headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
response = requests.get(url, headers=headers)
# soup = BeautifulSoup(response.text, "html.parser")
print(response.status_code)
# soup

200


In [292]:
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import traceback

def get_contents(yahoo):
    contents = []

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    for url in tqdm(yahoo['url']):
        try:
            response = requests.get(url, headers=headers, timeout=60)  # 1분 timeout 설정
            soup = BeautifulSoup(response.text, "html.parser")

            # First, try to find elements with the '.article-comm div' selector
            article_elements = soup.select('.article-body')
            
            
            # # # If not found, try the '.p-m-15' selector
            # if not article_elements:
            #     article_elements = soup.select('#fxs_article_content')

            # # If still not found, try the '.article-text' selector
            # if not article_elements:
            #     article_elements = soup.select('.p-m-15')

            if article_elements:
                content = article_elements[0].get_text()
            else:
                content = ''
            # print(article_elements)

            contents.append(content)
        except (requests.Timeout, Exception) as e:
            if isinstance(e, requests.Timeout):
                print(f"Request timed out for URL: {url}")
            else:
                print(f"Error occurred: {e}")
                traceback.print_exc()
            contents.append('')

    return contents


In [26]:
# import requests
# from bs4 import BeautifulSoup
# import re
# from tqdm import tqdm

# def get_contents(yahoo):
#     contents = []

#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

#     for url in tqdm(yahoo['url']):
#         try:
#             response = requests.get(url, headers=headers)
#             soup = BeautifulSoup(response.text, "html.parser")

#             # 모든 클래스 이름 중에서 ".col-*-offset-1" 패턴을 가진 것들을 찾는다
#             classes = set()
#             for tag in soup.find_all(True, {'class': True}):
#                 for class_name in tag['class']:
#                     if re.match(r'col-\w+-offset-1', class_name):
#                         classes.add(class_name)

#             # 해당 패턴을 가진 클래스들로부터 내용을 추출한다
#             content = []
#             for class_name in classes:
#                 content.extend(soup.select(f".{class_name}"))
            
#             cleaned_content = re.sub('<[^>]*>', '', ''.join(str(item) for item in content))
#             contents.append(cleaned_content)
#         except:
#             contents.append('')

#     return contents


In [294]:
head = TechTimes.head()
head['content_full'] = get_contents(head)
head

100%|██████████| 5/5 [00:03<00:00,  1.37it/s]
C:\Users\boyu571\AppData\Local\Temp\ipykernel_19872\1110637182.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  head['content_full'] = get_contents(head)


,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
3412,"{'id': None, 'name': 'Tech Times'}",NaN,Cryptocurrency Startup Tether Hacked: Attacker...,"Tether, a cryptocurrency startup, has lost $31...",http://www.techtimes.com/articles/215983/20171...,http://image.techtimes.com/data/images/full/31...,2017-11-21T14:57:36Z,How cryptocurrency is changing the world Nearl...,Tech Times,"\n\n\n\n\n\nLipin Pappachan, Tech Times\n ..."
6330,"{'id': None, 'name': 'Tech Times'}",NaN,Amazon Sumerian Is A VR App Creator For The VR...,Amazon has unveiled a web-based toolkit called...,http://www.techtimes.com/articles/216283/20171...,http://image.techtimes.com/data/images/full/31...,2017-11-29T11:18:04Z,The five craziest bids for Amazon's new HQ Whe...,Tech Times,"\n\n\n\n\n\nCarl Velasco, Tech Times\n ..."
8378,"{'id': None, 'name': 'Tech Times'}",NaN,Elon Musk Denies He Is Bitcoin Inventor Satosh...,Elon Musk as the creator of bitcoins steers up...,http://www.techtimes.com/articles/216422/20171...,http://image.techtimes.com/data/images/full/31...,2017-12-02T12:11:39Z,How cryptocurrency is changing the world The r...,Tech Times,"\n\n\n\n\n\nLipin Pappachan, Tech Times\n ..."
8801,"{'id': None, 'name': 'Tech Times'}",NaN,Cryptocurrency Owners Have Already Spent Almos...,Tech Times Cryptocurrency Owners Have Already ...,http://www.techtimes.com/articles/216570/20171...,http://image.techtimes.com/data/images/full/31...,2017-12-04T03:53:09Z,How cryptocurrency is changing the world Crypt...,Tech Times,"\n\n\n\n\n\nAaron Mamiit, Tech Times\n ..."
24214,"{'id': None, 'name': 'Tech Times'}",NaN,Bitcoin's Price Drops Sending Panic In The Cry...,Cryptocurrency is a volatile investment. With ...,http://www.techtimes.com/articles/217461/20171...,http://www.techtimes.com//image.techtimes.com/...,2017-12-25T05:17:47Z,How cryptocurrency is changing the world Bitco...,Tech Times,"\n\n\n\n\n\nJean-Pierre Chigne, Tech Times\n ..."


In [297]:
# Adding the content to the DataFrame
TechTimes['content_full'] = get_contents(TechTimes)
TechTimes

100%|██████████| 1337/1337 [12:17<00:00,  1.81it/s]
C:\Users\boyu571\AppData\Local\Temp\ipykernel_19872\2141718686.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TechTimes['content_full'] = get_contents(TechTimes)


,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
3412,"{'id': None, 'name': 'Tech Times'}",NaN,Cryptocurrency Startup Tether Hacked: Attacker...,"Tether, a cryptocurrency startup, has lost $31...",http://www.techtimes.com/articles/215983/20171...,http://image.techtimes.com/data/images/full/31...,2017-11-21T14:57:36Z,How cryptocurrency is changing the world Nearl...,Tech Times,"\n\n\n\n\n\nLipin Pappachan, Tech Times\n ..."
6330,"{'id': None, 'name': 'Tech Times'}",NaN,Amazon Sumerian Is A VR App Creator For The VR...,Amazon has unveiled a web-based toolkit called...,http://www.techtimes.com/articles/216283/20171...,http://image.techtimes.com/data/images/full/31...,2017-11-29T11:18:04Z,The five craziest bids for Amazon's new HQ Whe...,Tech Times,"\n\n\n\n\n\nCarl Velasco, Tech Times\n ..."
8378,"{'id': None, 'name': 'Tech Times'}",NaN,Elon Musk Denies He Is Bitcoin Inventor Satosh...,Elon Musk as the creator of bitcoins steers up...,http://www.techtimes.com/articles/216422/20171...,http://image.techtimes.com/data/images/full/31...,2017-12-02T12:11:39Z,How cryptocurrency is changing the world The r...,Tech Times,"\n\n\n\n\n\nLipin Pappachan, Tech Times\n ..."
8801,"{'id': None, 'name': 'Tech Times'}",NaN,Cryptocurrency Owners Have Already Spent Almos...,Tech Times Cryptocurrency Owners Have Already ...,http://www.techtimes.com/articles/216570/20171...,http://image.techtimes.com/data/images/full/31...,2017-12-04T03:53:09Z,How cryptocurrency is changing the world Crypt...,Tech Times,"\n\n\n\n\n\nAaron Mamiit, Tech Times\n ..."
24214,"{'id': None, 'name': 'Tech Times'}",NaN,Bitcoin's Price Drops Sending Panic In The Cry...,Cryptocurrency is a volatile investment. With ...,http://www.techtimes.com/articles/217461/20171...,http://www.techtimes.com//image.techtimes.com/...,2017-12-25T05:17:47Z,How cryptocurrency is changing the world Bitco...,Tech Times,"\n\n\n\n\n\nJean-Pierre Chigne, Tech Times\n ..."
...,...,...,...,...,...,...,...,...,...,...
646798,"{'id': None, 'name': 'Tech Times'}",John Lopez,South Korea Invests $18 Million in New 'Metave...,The South Korean Ministry of Science and ICT h...,https://www.techtimes.com/articles/288860/2023...,https://1734811051.rsc.cdn77.org/data/images/f...,2023-03-12T18:09:36Z,South Korea's Ministry of Science and ICT rece...,Tech Times,"\n\n\n\n\n\nJohn Lopez, Tech Times\n ..."
646976,"{'id': None, 'name': 'Tech Times'}",Urian B.,Afun Interactive Gets $7.6 Million in Series A...,Afun Interactive recently raised $7.6 million ...,https://www.techtimes.com/articles/288877/2023...,https://1734811051.rsc.cdn77.org/data/images/f...,2023-03-13T05:10:00Z,"Afun Interactive, a Korean startup specializin...",Tech Times,"\n\n\n\n\n\nUrian B., Tech Times\n ..."
648919,"{'id': None, 'name': 'Tech Times'}",John Lopez,Cybercriminals' Favorite Crypto Platform ChipM...,Criminals use tools like ChipMixer to make lau...,https://www.techtimes.com/articles/289005/2023...,https://1734811051.rsc.cdn77.org/data/images/f...,2023-03-15T17:10:00Z,"In a landmark effort against cybercrime, globa...",Tech Times,"\n\n\n\n\n\nJohn Lopez, Tech Times\n ..."
652254,"{'id': None, 'name': 'Tech Times'}",John Lopez,GameStop's Quarterly Profit Results in 40% Sur...,GameStop's shares rose 40% in premarket trade ...,https://www.techtimes.com/articles/289334/2023...,https://1734811051.rsc.cdn77.org/data/images/f...,2023-03-22T12:28:05Z,GameStop shares soared 40% in premarket trade ...,Tech Times,"\n\n\n\n\n\nJohn Lopez, Tech Times\n ..."


In [285]:
head['url'].iloc[3]

'http://www.thestar.com.my/business/business-news/2017/11/13/bitcoin-tumble-erases-us$38b-as-rival-cryptocurrency-gains/'

In [296]:
head['content_full'].iloc[0]

"\n\n\n\n\n\nLipin Pappachan, Tech Times\n                    \n                21 November 2017, 09:11 am\n\n\n\nNearly $31 million worth of bitcoin currency was wiped off from cryptocurrency\xa0startup Tethers' Treasury account. The startup is in business with bitcoin exchanges and the hacker was able to move the bitcoin to an unauthorized account.\nTether has released the details of the attack and precautionary measures on its website for its users.\nDetails Of The Attack That Stole $31 Million\nOn Nov. 19, 2017, the Treasury account of Tether was compromised by a hacker. The bitcoins were maliciously transferred from Tether to the following address: 16tg2RJuEPtZooy18Wxn2me2RhUdC94N7r by the hacker. Although the fund address has now been flagged, the curtailment of further damage will depend on how the startup is able to control the spread of this fund in the broader ecosystem. Tether has requested people who trade in bitcoins to not to purchase USDT tokens from the above address.\n

In [298]:
empty_rows = TechTimes[TechTimes['content_full'] == '']
print(len(empty_rows))

15


In [289]:
empty_rows['url'].iloc[40]

'http://www.thestar.com.my/business/business-news/2019/06/04/crypto-pioneer-justin-sun-pays-us$4pt57m-for-warren-buffett-lunch/'

<!-- # empty_rows['url'].iloc[0] -->

In [299]:
TechTimes.to_csv(r'C:\Users\boyu571\boyu571_Github\01_Kakaobank_SKKU_Research_23\data\foreign_news_data\TechTimes_76.csv', encoding='utf-8-sig')

In [ ]:
#article-13715 > div.post__content-wrapper > div.post-content